In [1]:
import numpy as np

from jitcsde import jitcsde, y
import math

import plotly.express as px

## Hurwitz matrix construction

In [2]:
## Algunos ejemplos
# Diagonal
A_mat = np.matrix([
        [-1.0,0.0, -1.0],
        [0.5, -2.0, 0.0],
        [0.0, -1.0, -1.0]
    ])


## Numerical solver

In [3]:
from numpy import diff


N = len(A_mat)
diff_sys = [sum([A_mat[i,j]*y(j) for j in range(N)]) for i in range(N)]
print(diff_sys)
noise_term = [1. for _ in range(N)]
SDE = jitcsde(diff_sys, noise_term)

initial_state = np.array([0.0  for _ in range(N)])
SDE.set_initial_value(initial_state, 0.0)

data = []
count = 0
for time in np.arange(0.0, 10000.0, 0.01):
    if count%100 == 0:
        data.append(SDE.integrate(time))
    count += 1
data = np.array(data)

[-1.0*y(0) - 1.0*y(2), 0.5*y(0) - 2.0*y(1), -1.0*y(1) - 1.0*y(2)]
Generating, compiling, and loading C code.
Using default integration parameters.


In [19]:
fig = px.scatter(x = data[:-1,0], y = (data[1:,0]-data[:-1,0]))

fig.show()

In [6]:
np.corrcoef(data[:,1],data[:,2])

array([[ 1.        , -0.36584391],
       [-0.36584391,  1.        ]])

In [5]:
# source = data[:-1,1]
# destination = (data[1:,0]-data[:-1,0])

source = data[:,0]
destination = data[:,1]
fig = px.scatter(x = source, y = destination, trendline="ols")
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_traces(marker=dict(size=5, color="Black"),
                  selector=dict(mode='markers'))
fig.show()

In [8]:
results = px.get_trendline_results(fig)
print(results)

results.px_fit_results.iloc[0].summary()

                                      px_fit_results
0  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     802.1
Date:                Wed, 12 Oct 2022   Prob (F-statistic):          5.65e-161
Time:                        09:57:29   Log-Likelihood:                -2907.1
No. Observations:                4000   AIC:                             5818.
Df Residuals:                    3998   BIC:                             5831.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0045      0.008     -0.565      0.572      -0.020       0.011
x1             0.2390      0.008     28.322      0.000       0.222       0.256
==============================================================================
Omnibus:                        2.758   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.252   Jarque-Bera (JB):                2.592
Skew:                           0.010   Prob(JB):                        0.274
Kurtosis:                       2.877   Cond. No.                         1.07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Transfer entropy estimation with JIDT

In [5]:
import sys
import os
import jpype

In [16]:
## TE condicionado a la tercer dimensión

# JIDT jar library path
jarLocation = "JIDT/infodynamics.jar"
# Start the JVM
if not jpype.isJVMStarted():
    jpype.startJVM("/Library/Java/JavaVirtualMachines/jdk-18.0.1.1.jdk/Contents/Home/lib/libjli.dylib", "-ea", "-Djava.class.path=" + jarLocation)

TE_vals = []

# Construct the calculator:
calcClass = jpype.JPackage("infodynamics.measures.continuous.kraskov").ConditionalTransferEntropyCalculatorKraskov
calc = calcClass()
# Set any properties to non-default values:
# calc.setProperty("AUTO_EMBED_METHOD", "RAGWITZ_DEST_ONLY")
# calc.setProperty("NUM_THREADS", "USE_ALL")

third_excluded = {
    "01" : 2,
    "12" : 0,
    "02" : 1
}

G_te = np.matrix([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]])

# Compute for all pairs:
for s in range(N):
    for d in range(N):
        # For each source-dest pair:
        if (s == d):
            continue
        # source = data[:-1,s]
        # destination = (data[1:,d]-data[:-1,d])
        source = data[:,s]
        destination = data[:,d]
        conditional = data[:,third_excluded["".join(sorted([str(s),str(d)]))]]

        # Initialise the calculator for (re-)use:
        calc.initialise()
        # Supply the sample data:
        calc.setObservations(source, destination, conditional)
        # Compute the estimate:
        result = calc.computeAverageLocalOfObservations()
        # Compute the (statistical significance via) null distribution empirically (e.g. with 100 permutations):
        measDist = calc.computeSignificance(100)

        pVal = measDist.pValue

        if pVal <= 0.05:
            G_te[d,s] = 1-pVal
            print("{}->{} : {} , {}".format(s,d,G_te[d,s], pVal))

        print("Transfer entropy ({}->{}): {:.4f} nats (std = {:.4f} ± {:.4f}, p < {:.4f}). Correlation = {}".format(s, d, result, measDist.getMeanOfDistribution(), measDist.getStdOfDistribution(), pVal, np.corrcoef(source, destination)[0,1]))
        TE_vals.append(result)

fig = px.imshow(G_te)
fig.show()

0->1 : 1.0 , 0.0
Transfer entropy (0->1): 0.0191 nats (std = 0.0008 ± 0.0061, p < 0.0000). Correlation = 0.402385867043657
Transfer entropy (0->2): 0.0012 nats (std = -0.0007 ± 0.0054, p < 0.4000). Correlation = -0.5600272297916652
Transfer entropy (1->0): 0.0039 nats (std = -0.0002 ± 0.0052, p < 0.2300). Correlation = 0.40238586704365703
1->2 : 0.99 , 0.01
Transfer entropy (1->2): 0.0129 nats (std = -0.0002 ± 0.0062, p < 0.0100). Correlation = -0.39431743880800596
2->0 : 1.0 , 0.0
Transfer entropy (2->0): 0.0559 nats (std = -0.0004 ± 0.0062, p < 0.0000). Correlation = -0.5600272297916652
Transfer entropy (2->1): 0.0095 nats (std = 0.0006 ± 0.0056, p < 0.0700). Correlation = -0.3943174388080059


In [16]:
fig = px.imshow(np.abs(np.sign(A_mat)))
fig.show()

In [11]:
## Directamente TE

# JIDT jar library path
jarLocation = "JIDT/infodynamics.jar"
# Start the JVM
if not jpype.isJVMStarted():
    jpype.startJVM("/Library/Java/JavaVirtualMachines/jdk-18.0.1.1.jdk/Contents/Home/lib/libjli.dylib", "-ea", "-Djava.class.path=" + jarLocation)

TE_vals = []

# Construct the calculator:
calcClass = jpype.JPackage("infodynamics.measures.continuous.kraskov").TransferEntropyCalculatorKraskov
calc = calcClass()
# Set any properties to non-default values:
# calc.setProperty("AUTO_EMBED_METHOD", "RAGWITZ_DEST_ONLY")
# calc.setProperty("NUM_THREADS", "USE_ALL")

# Compute for all pairs:
for s in range(N):
    for d in range(N):
        # For each source-dest pair:
        if (s == d):
            continue
        # source = data[:-1,s]
        # destination = (data[1:,d]-data[:-1,d])
        source = data[:,s]
        destination = data[:,d]

        # Initialise the calculator for (re-)use:
        calc.initialise()
        # Supply the sample data:
        calc.setObservations(source, destination)
        # Compute the estimate:
        result = calc.computeAverageLocalOfObservations()
        # Compute the (statistical significance via) null distribution empirically (e.g. with 100 permutations):
        measDist = calc.computeSignificance(100)

        print("Transfer entropy ({}->{}): {:.4f} nats (std = {:.4f} ± {:.4f}, p < {:.4f}). Correlation = {}".format(s, d, result, measDist.getMeanOfDistribution(), measDist.getStdOfDistribution(), measDist.pValue, np.corrcoef(source, destination)[0,1]))
        TE_vals.append(result)

Transfer entropy (0->1): 0.0430 nats (std = -0.0007 ± 0.0084, p < 0.0000). Correlation = 0.40878119885987546
Transfer entropy (0->2): 0.0024 nats (std = 0.0005 ± 0.0090, p < 0.3700). Correlation = -0.533618012357621
Transfer entropy (1->0): 0.0160 nats (std = 0.0010 ± 0.0087, p < 0.0700). Correlation = 0.40878119885987546
Transfer entropy (1->2): 0.0103 nats (std = -0.0001 ± 0.0087, p < 0.1300). Correlation = -0.3658439089659783
Transfer entropy (2->0): 0.0604 nats (std = 0.0014 ± 0.0085, p < 0.0000). Correlation = -0.533618012357621
Transfer entropy (2->1): 0.0103 nats (std = -0.0004 ± 0.0087, p < 0.1200). Correlation = -0.36584390896597824


In [12]:
## TE modificada como I(x_i; Δx_j | x_j). Es decir, la información que tiene la posición x_i sobre el cambio en la dirección x_j, que no se puede obtener de saber la posición x_j

# JIDT jar library path
jarLocation = "JIDT/infodynamics.jar"
# Start the JVM
if not jpype.isJVMStarted():
    jpype.startJVM("/Library/Java/JavaVirtualMachines/jdk-18.0.1.1.jdk/Contents/Home/lib/libjli.dylib", "-ea", "-Djava.class.path=" + jarLocation)

TE_vals = []

# Construct the calculator:
calcClass = jpype.JPackage("infodynamics.measures.continuous.kraskov").ConditionalMutualInfoCalculatorMultiVariateKraskov1
calc = calcClass()
# Set any properties to non-default values:
# calc.setProperty("AUTO_EMBED_METHOD", "RAGWITZ_DEST_ONLY")
# calc.setProperty("NUM_THREADS", "USE_ALL")

# Compute for all pairs:
for s in range(N):
    for d in range(N):
        # For each source-dest pair:
        if (s == d):
            continue
        source = data[:-1,s]
        destination = (data[1:,d]-data[:-1,d])
        conditional = data[:-1,d]

        # Initialise the calculator for (re-)use:
        calc.initialise()
        # Supply the sample data:
        calc.setObservations(source, destination, conditional)
        # Compute the estimate:
        result = calc.computeAverageLocalOfObservations()
        # Compute the (statistical significance via) null distribution empirically (e.g. with 100 permutations):
        measDist = calc.computeSignificance(1000)

        print("Transfer entropy ({}->{}): {:.4f} nats (std = {:.4f} ± {:.4f}, p < {:.4f})".format(s, d, result, measDist.getMeanOfDistribution(), measDist.getStdOfDistribution(), measDist.pValue))
        TE_vals.append(result)

Transfer entropy (0->1): 0.0415 nats (std = -0.0002 ± 0.0094, p < 0.0000)
Transfer entropy (0->2): 0.0067 nats (std = -0.0003 ± 0.0095, p < 0.2240)
Transfer entropy (1->0): 0.0208 nats (std = 0.0002 ± 0.0094, p < 0.0140)
Transfer entropy (1->2): 0.0256 nats (std = 0.0000 ± 0.0094, p < 0.0030)


KeyboardInterrupt: 

In [ ]:

# JIDT jar library path
jarLocation = "JIDT/infodynamics.jar"
# Start the JVM
if not jpype.isJVMStarted():
    jpype.startJVM("/Library/Java/JavaVirtualMachines/jdk-18.0.1.1.jdk/Contents/Home/lib/libjli.dylib", "-ea", "-Djava.class.path=" + jarLocation)

TE_vals = []

# Construct the calculator:
calcClass = jpype.JPackage("infodynamics.measures.continuous.kraskov").MutualInfoCalculatorMultiVariateKraskov1
calc = calcClass()

# Compute for all nodes:
for s in range(N):
    source = data[:-1,s]
    destination = (data[1:,s]-data[:-1,s])

    # Initialise the calculator for (re-)use:
    calc.initialise()
    # Supply the sample data:
    calc.setObservations(source, destination)
    # Compute the estimate:
    result = calc.computeAverageLocalOfObservations()
    # Compute the (statistical significance via) null distribution empirically (e.g. with 100 permutations):
    measDist = calc.computeSignificance(100)

    print("Mutual info (x_{}->Δx_{}): {:.4f} nats (std = {:.4f} ± {:.4f}, p < {:.4f})".format(s, s, result, measDist.getMeanOfDistribution(), measDist.getStdOfDistribution(), measDist.pValue, 100))
    TE_vals.append(result)

Mutual info (x_0->Δx_0): 0.0189 nats (std = 0.0009 ± 0.0103, p < 0.0500)
Mutual info (x_1->Δx_1): 0.0204 nats (std = 0.0001 ± 0.0103, p < 0.0300)
Mutual info (x_2->Δx_2): 0.0076 nats (std = 0.0017 ± 0.0103, p < 0.2400)
